For Pretraining Temperature: You could experiment with different values of pretraining_tp and observe how it affects the behavior of the model during training or inference, especially if sampling methods are used.

Dataset: https://huggingface.co/datasets/tatsu-lab/alpaca
Alpaca is a dataset of 52,000 instructions and demonstrations generated by OpenAI's text-davinci-003 engine. This instruction data can be used to conduct instruction-tuning for language models and make the language model follow instruction better.

The authors built on the data generation pipeline from Self-Instruct framework and made the following modifications:

The text-davinci-003 engine to generate the instruction data instead of davinci.
A new prompt was written that explicitly gave the requirement of instruction generation to text-davinci-003.
Much more aggressive batch decoding was used, i.e., generating 20 instructions at once, which significantly reduced the cost of data generation.
The data generation pipeline was simplified by discarding the difference between classification and non-classification instructions.
Only a single instance was generated for each instruction, instead of 2 to 3 instances as in Self-Instruct.
This produced an instruction-following dataset with 52K examples obtained at a much lower cost (less than $500). In a preliminary study, the authors also found that the 52K generated data to be much more diverse than the data released by Self-Instruct.

Supported Tasks and Leaderboards
The Alpaca dataset designed for instruction training pretrained language models.

Languages
The data in Alpaca are in English (BCP-47 en).

Dataset Structure
Data Instances
An example of "train" looks as follows:



In [1]:
"""{
    "instruction": "Create a classification task by clustering the given list of items.",
    "input": "Apples, oranges, bananas, strawberries, pineapples",
    "output": "Class 1: Apples, Oranges\nClass 2: Bananas, Strawberries\nClass 3: Pineapples",
    "text": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a classification task by clustering the given list of items.\n\n### Input:\nApples, oranges, bananas, strawberries, pineapples\n\n### Response:\nClass 1: Apples, Oranges\nClass 2: Bananas, Strawberries\nClass 3: Pineapples",
}"""

'{\n    "instruction": "Create a classification task by clustering the given list of items.",\n    "input": "Apples, oranges, bananas, strawberries, pineapples",\n    "output": "Class 1: Apples, Oranges\nClass 2: Bananas, Strawberries\nClass 3: Pineapples",\n    "text": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a classification task by clustering the given list of items.\n\n### Input:\nApples, oranges, bananas, strawberries, pineapples\n\n### Response:\nClass 1: Apples, Oranges\nClass 2: Bananas, Strawberries\nClass 3: Pineapples",\n}'

In [4]:
!pip install -q ipython-autotime
%load_ext autotime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.3 MB/s eta 0:00:00
time: 315 µs (started: 2023-09-09 16:20:38 +00:00)


In [5]:
!pip install -q git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.5 MB/s eta 0:00:00
time: 27.3 s (started: 2023-09-09 16:20:38 +00:00)


In [6]:
!pip install -q git+https://github.com/huggingface/datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
time: 15.1 s (started: 2023-09-09 16:21:05 +00:00)


In [7]:
!pip install -q git+https://github.com/huggingface/peft

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.0 MB/s eta 0:00:00
time: 13.8 s (started: 2023-09-09 16:21:20 +00:00)


In [8]:
!pip install -q git+https://github.com/huggingface/accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
time: 13 s (started: 2023-09-09 16:21:34 +00:00)


In [9]:
!pip install -q --upgrade bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.9 MB/s eta 0:00:00
time: 8.34 s (started: 2023-09-09 16:21:47 +00:00)


In [10]:
!pip install -q git+https://github.com/huggingface/trl

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
time: 12.9 s (started: 2023-09-09 16:21:56 +00:00)


In [11]:
!pip install -q --upgrade safetensors

time: 5.02 s (started: 2023-09-09 16:22:09 +00:00)


In [12]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("tatsu-lab/alpaca", split="train")





Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

time: 4.03 s (started: 2023-09-09 16:22:14 +00:00)


In [13]:
print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])

dataset size: 52002
{'instruction': 'Explain why this product could be used as an affordable gift', 'input': 'A USB car charger', 'output': 'A USB car charger is an affordable and practical gift for anyone who spends a lot of time on the road. It provides a convenient way to charge devices in the car, eliminating the need for an extra power outlet. The charger is compact and lightweight, making it easy to store and transport. Moreover, the price of USB car chargers is usually quite affordable, making it a great gift for any occasion.', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nExplain why this product could be used as an affordable gift\n\n### Input:\nA USB car charger\n\n### Response:\nA USB car charger is an affordable and practical gift for anyone who spends a lot of time on the road. It provides a convenient way to charge devices in the c

In [14]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})

time: 2.18 ms (started: 2023-09-09 16:22:18 +00:00)


In [15]:
# Rename columns in the dataset
dataset = dataset.rename_column('input', 'context')
dataset = dataset.rename_column('output', 'response')
dataset = dataset.rename_column('text', 'category')

time: 4.82 ms (started: 2023-09-09 16:22:18 +00:00)


In [16]:
dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 52002
})

time: 1.86 ms (started: 2023-09-09 16:22:18 +00:00)


To instruct tune our model, we need to convert our structured examples into a collection of tasks described via instructions. We define a formatting_function that takes a sample and returns a string with our format instruction.

In [17]:
def format_instruction(sample):
	return f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{sample['response']}

### Response:
{sample['instruction']}
"""


time: 409 µs (started: 2023-09-09 16:22:18 +00:00)


# **Testing Formatting Function on a Random Example**

In [18]:
from random import randrange

print(format_instruction(dataset[randrange(len(dataset))]))



### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
The human visual spectrum has a range of wavelengths from about 390 nanometers to about 700 nanometers. This range of light waves is known as visible light, and it corresponds to the colors of the rainbow--red, orange, yellow, green, blue, indigo and violet.

### Response:
What is the range of the human visual spectrum?

time: 819 µs (started: 2023-09-09 16:22:18 +00:00)


In [19]:
print(dataset[0].keys())


dict_keys(['instruction', 'context', 'response', 'category'])
time: 647 µs (started: 2023-09-09 16:22:18 +00:00)


QLoRA is a new technique to reduce the memory footprint of large language models during finetuning, without sacrificing performance. The TL;DR; of how QLoRA works is:

Quantize the pre-trained model to 4 bits and freeze it.
Attach small, trainable adapter layers. (LoRA)
Finetune only the adapter layers while using the frozen quantized model for context.

Flash Attention is a an method that reorders the attention computation and leverages classical techniques (tiling, recomputation) to significantly speed it up and reduce memory usage from quadratic to linear in sequence length. It is based on the paper "FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness". The TL;DR; accelerates training up to 3x. Learn more at FlashAttention. Flash Attention is currently only available for Ampere (A10, A40, A100, ...) & Hopper (H100, ...) GPUs.

In [20]:
"""!python -c "import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'"
!pip install ninja packaging
!MAX_JOBS=4 pip install flash-attn --no-build-isolation
"""

'!python -c "import torch; assert torch.cuda.get_device_capability()[0] >= 8, \'Hardware not supported for Flash Attention\'"\n!pip install ninja packaging\n!MAX_JOBS=4 pip install flash-attn --no-build-isolation\n'

time: 1.76 ms (started: 2023-09-09 16:22:18 +00:00)


# **Importing Necessary Libraries**

In [21]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

time: 4.16 s (started: 2023-09-09 16:22:18 +00:00)


# **Flash Attention Flag and Checking Device Capability and Patching**

In [22]:
use_flash_attention = False

"""if torch.cuda.get_device_capability()[0] >= 8:
    from utils.llama_patch import replace_attn_with_flash_attn
    print("Using flash attention")
    replace_attn_with_flash_attn()
    use_flash_attention = True"""

'if torch.cuda.get_device_capability()[0] >= 8:\n    from utils.llama_patch import replace_attn_with_flash_attn\n    print("Using flash attention")\n    replace_attn_with_flash_attn()\n    use_flash_attention = True'

time: 2.33 ms (started: 2023-09-09 16:22:22 +00:00)


# **Model ID Specification**

In [23]:
# Hugging Face model id
model_id = "NousResearch/Llama-2-7b-hf"

time: 268 µs (started: 2023-09-09 16:22:22 +00:00)


# **BitsAndBytesConfig Setup**

In [24]:
# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

time: 2.08 ms (started: 2023-09-09 16:22:22 +00:00)


In [25]:
import torch
torch.cuda.empty_cache()


time: 321 µs (started: 2023-09-09 16:22:22 +00:00)


# **Loading Model and Tokenizer**

In [26]:
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache=False, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

time: 1min 8s (started: 2023-09-09 16:22:22 +00:00)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [27]:
model.config

LlamaConfig {
  "_name_or_path": "NousResearch/Llama-2-7b-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.3

time: 3.67 ms (started: 2023-09-09 16:23:30 +00:00)


In [28]:
model.config.pretraining_tp = 1

time: 308 µs (started: 2023-09-09 16:23:30 +00:00)


In [29]:
model.config

LlamaConfig {
  "_name_or_path": "NousResearch/Llama-2-7b-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.3

time: 3.53 ms (started: 2023-09-09 16:23:30 +00:00)


In [30]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

time: 979 ms (started: 2023-09-09 16:23:30 +00:00)


In [31]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)


time: 8.96 s (started: 2023-09-09 16:23:31 +00:00)


In [32]:
model.config

LlamaConfig {
  "_name_or_path": "NousResearch/Llama-2-7b-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.3

time: 3.6 ms (started: 2023-09-09 16:23:40 +00:00)


# **Instantiate LoRAConfig and Prepare Model for Training**

In [33]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="llama-7-int4-alpaca-normal-attention-tp-1",
    num_train_epochs=2,
    per_device_train_batch_size=6,
    optim="paged_adamw_32bit",
    logging_steps=200,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    lr_scheduler_type="constant",
    disable_tqdm=True # disable tqdm since with packing values are in correct
)

time: 4.24 ms (started: 2023-09-09 16:23:40 +00:00)


In [34]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)


time: 230 ms (started: 2023-09-09 16:23:40 +00:00)


In [35]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.9377, 'learning_rate': 0.0002, 'epoch': 0.02}
{'loss': 0.8898, 'learning_rate': 0.0002, 'epoch': 0.05}
{'loss': 0.8708, 'learning_rate': 0.0002, 'epoch': 1.01}
{'loss': 0.8654, 'learning_rate': 0.0002, 'epoch': 1.03}
{'loss': 0.864, 'learning_rate': 0.0002, 'epoch': 1.06}
{'train_runtime': 6819.2035, 'train_samples_per_second': 15.252, 'train_steps_per_second': 2.542, 'train_loss': 0.884767323675174, 'epoch': 1.06}


TrainOutput(global_step=1042, training_loss=0.884767323675174, metrics={'train_runtime': 6819.2035, 'train_samples_per_second': 15.252, 'train_steps_per_second': 2.542, 'train_loss': 0.884767323675174, 'epoch': 1.06})

time: 1h 53min 39s (started: 2023-09-09 16:23:41 +00:00)


In [37]:
from huggingface_hub import notebook_login
notebook_login()

time: 18.2 ms (started: 2023-09-09 18:29:52 +00:00)


In [39]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.16k [00:00<?, ?B/s]

'https://huggingface.co/DrishtiSharma/llama-7-int4-alpaca-normal-attention-tp-1/tree/main/'

time: 7.41 s (started: 2023-09-09 18:31:36 +00:00)


# **Test Model and run Inference**

In [40]:
if use_flash_attention:
    # unpatch flash attention
    from utils.llama_patch import unplace_flash_attn_with_attn
    unplace_flash_attn_with_attn()

import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer


# load base LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


time: 14.9 s (started: 2023-09-09 18:31:44 +00:00)


In [41]:
sample = dataset[randrange(len(dataset))]

prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{sample['response']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)

print(f"Prompt:\n{sample['response']}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample['instruction']}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Prompt:
The COVID-19 pandemic had a major impact on the food industry in 20

Generated instruction:
Generate a new sentence that starts with the phrase "The COVID-19 pandemic had a major impact on the food industry in".

Ground truth:
Analyze how 2020 impacted the food industry.
time: 3.61 s (started: 2023-09-09 18:31:59 +00:00)


In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.9378, 'learning_rate': 0.0002, 'epoch': 0.02}
{'loss': 0.8898, 'learning_rate': 0.0002, 'epoch': 0.05}
{'loss': 0.8709, 'learning_rate': 0.0002, 'epoch': 1.01}
{'loss': 0.8655, 'learning_rate': 0.0002, 'epoch': 1.03}
{'loss': 0.864, 'learning_rate': 0.0002, 'epoch': 1.06}
{'train_runtime': 6840.4904, 'train_samples_per_second': 15.204, 'train_steps_per_second': 2.534, 'train_loss': 0.8848599161159054, 'epoch': 1.06}


TrainOutput(global_step=1042, training_loss=0.8848599161159054, metrics={'train_runtime': 6840.4904, 'train_samples_per_second': 15.204, 'train_steps_per_second': 2.534, 'train_loss': 0.8848599161159054, 'epoch': 1.06})

time: 1h 54min (started: 2023-09-09 15:55:29 +00:00)


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

time: 18.2 ms (started: 2023-09-09 18:05:46 +00:00)


In [42]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
)

# Merge LoRA and base model
merged_model = model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")

# push merged model to the hub
merged_model.push_to_hub("DrishtiSharma/llama-2-7b-int4-alpaca-normal-attention-tp-1-merged")
tokenizer.push_to_hub("DrishtiSharma/llama-2-7b-int4-alpaca-normal-attention-tp-1-merged")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/DrishtiSharma/llama-2-7b-int4-alpaca-normal-attention-tp-1-merged/commit/a191fdaedacda70693ccfcc3aa24b54a8a1d7f56', commit_message='Upload tokenizer', commit_description='', oid='a191fdaedacda70693ccfcc3aa24b54a8a1d7f56', pr_url=None, pr_revision=None, pr_num=None)

time: 11min 20s (started: 2023-09-09 18:33:12 +00:00)
